In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pandas.tseries.offsets import DateOffset
import openpyxl as op
from tqdm.auto import tqdm
import plotly.graph_objs as go
import time
import plotly.express as px
import warnings
warnings.filterwarnings("ignore", message="No frequency information was")
warnings.filterwarnings("ignore", message="Maximum Likelihood optimization failed to ")

instances = pd.read_csv("../instances/instances.csv", index_col=0).to_numpy().reshape(14)

In [2]:
INSTANCE = 0

In [3]:
data_series = pd.read_csv("../instances/{n}/series_{n}.csv".format(n=instances[INSTANCE]))
data_series['local_15min'] = pd.to_datetime(data_series['local_15min'])
data_series.index = data_series['local_15min']
data_series.drop(columns=['local_15min'], inplace=True)
data_demand = data_series[['nonshiftable']]

In [4]:
data_demand_hourly = data_demand.resample("H").mean()
data_demand_hourly.head()

,nonshiftable
local_15min,
2019-05-01 00:00:00,0.065125
2019-05-01 01:00:00,0.043813
2019-05-01 02:00:00,0.067750
2019-05-01 03:00:00,0.099062
2019-05-01 04:00:00,0.177687


In [19]:
start_date = "2019-05-01"
end_date = "2019-07-01"
training_data = data_demand_hourly[(data_demand_hourly.index >= start_date) & (data_demand_hourly.index<end_date)]

In [8]:
def sarima_model(training_data, order, seasonal_order, timesteps, days):
    
    k = timesteps*days

    # Inicializar df con la información
    df = pd.DataFrame()

    # Calcular cantidad de días a pronósticar
    days_forecast = training_data.index[-1]-training_data.index[k-1]
    days_forecast = days_forecast.days

    for i in range(days_forecast):
        
        # Training set
        train = training_data[timesteps*i:k+timesteps*i]
        model = sm.tsa.SARIMAX(train['nonshiftable'].asfreq("60min"), order = order,seasonal_order=seasonal_order, freq=train.index.inferred_freq).fit(disp=False)

        for j in range(timesteps):

            # Prediction
            forecast_dates=[train.index[-1]+ DateOffset(minutes=60*j) + DateOffset(minutes=60*x)for x in range(1,timesteps+1-j)]
            df_forecast = training_data.loc[forecast_dates].copy()
            forecast = model.get_prediction(start = forecast_dates[0], end = forecast_dates[-1])
            forecast_mean = forecast.predicted_mean
            forecast_mean.index = forecast_dates
            forecast_U = forecast.predicted_mean + forecast.se_mean
            forecast_L = forecast.predicted_mean - forecast.se_mean
            forecast_U.index = forecast_dates
            forecast_L.index = forecast_dates

            # MSE
            df_forecast['forecast'] = forecast_mean
            #df_forecast['MSE'] = (df_forecast['forecast']-df_forecast['p'])**2
            df_forecast['Upper bound'] = forecast_U
            df_forecast['Lower bound'] = forecast_L

            # Update model with new data
            new_data = train.index[-1]+DateOffset(minutes=60*(j+1))
            model = model.extend(training_data['nonshiftable'][new_data:new_data].asfreq("60min"))

            # Update dataframe
            df_forecast['From'] = train.index[-1] + DateOffset(minutes=60*j)
            df_forecast['Day'] = train.index[-1] + DateOffset(minutes=60)
            df = pd.concat([df, df_forecast.reset_index(names='local_15min')])


        #print(str(i)+"|"+str(days_forecast-1),'train:', train.index[0], '->', train.index[-1], '|forecast:', forecast_dates[0], '->', forecast_dates[-1], '|mse:', df_forecast['MSE'].mean())

    return df

In [7]:
def auto_sarima(training_data, order_limits, timesteps, days, ruta):

    # Crear libro de excel
    book = op.Workbook()
    sheet = book.active
    sheet.title = 'Resultados'
    sheet.cell(3,1).value = 'p'
    sheet.cell(3,2).value = 'd'
    sheet.cell(3,3).value = 'q'
    sheet.cell(3,4).value = 'sp'
    sheet.cell(3,5).value = 'sd'
    sheet.cell(3,6).value = 'sq'
    sheet.cell(3,7).value = 's'
    sheet.cell(3,8).value = 'mse_mean'
    sheet.cell(3,9).value = 'mse_sd'

    # Inicializar lista con el mse de cada modelo y el contador
    mse_models = {}
    c = 3
    for p in order_limits['p']:
        for d in order_limits['d']:
            for q in order_limits['q']:
                for s in order_limits['s']:
                    for sp in order_limits['sp']:
                        for sd in order_limits['sd']:
                            for sq in order_limits['sq']:

                                if(s == 0 and sp+sd+sq!=0):
                                    continue
                          
                            
                                #print(" Model:", "({}, {}, {}), s({}, {}, {}, {})".format(p,d,q,sp,sd,sq,s))
                                time.sleep(0.1)
                                df = sarima_model(training_data=training_data, order=(p,d,q), seasonal_order=(sp,sd,sq,s), timesteps=timesteps, days=days)
                                df['mse'] = (df['nonshiftable']-df['forecast'])**2
                                mse_mean = df['mse'].mean()
                                #print(mse_mean)
                                mse_sd = df['mse'].var()**(1/2)
                                mse_models["({}, {}, {}), s({}, {}, {}, {})".format(p,d,q,sp,sd,sq,s)] = mse_mean
                                c = c+1
                                sheet.cell(c,1).value = p
                                sheet.cell(c,2).value = d
                                sheet.cell(c,3).value = q
                                sheet.cell(c,4).value = sp
                                sheet.cell(c,5).value = sd
                                sheet.cell(c,6).value = sq
                                sheet.cell(c,7).value = s
                                sheet.cell(c,8).value = mse_mean
                                sheet.cell(c,9).value = mse_sd

                                book.save(ruta)

    # best model
    sheet.cell(1,1).value = 'Mejor modelo:'
    sheet.cell(1,2).value = min(mse_models, key=mse_models.get)
    sheet.cell(1,3).value = min(mse_models.values())

    book.save(ruta)

In [ ]:
# Parámetros
timesteps = 24
days = 31 # 5 dias de observación
ruta = 'resultados/60min/sarima/sarima2_i{}.xlsx'.format(INSTANCE)

order_limits = {
    'p' : range(1,1+ 2),
    'd' : range(1,1+ 1),
    'q' : range(1,1+ 2),
    'sp' : range(1,1+ 1),
    'sd' : range(0,1+ 0),
    'sq' : range(1,1+ 1),
    's': [24]
}

auto_sarima(training_data=training_data, order_limits=order_limits, timesteps=timesteps, days=days, ruta=ruta)
    

In [ ]:
for i in tqdm(range(len(instances))):

    if i<2:
        continue

    data_series = pd.read_csv("../instances/{n}/series_{n}.csv".format(n=instances[i]))
    data_series['local_15min'] = pd.to_datetime(data_series['local_15min'])
    data_series.index = data_series['local_15min']
    data_series.drop(columns=['local_15min'], inplace=True)
    data_demand = data_series[['nonshiftable']]
    data_demand_hourly = data_demand.resample("H").mean()

    start_date = "2019-05-17"
    end_date = "2019-08-01"
    training_data = data_demand_hourly[(data_demand_hourly.index >= start_date) & (data_demand_hourly.index<end_date)]

    timesteps = 24
    days = 15 # 5 dias de observación
    ruta = 'resultados/60min/sarima_15/sarima_i{}.xlsx'.format(i)
    order_limits = {
        'p' : range(1,1+ 2),
        'd' : range(0,1+ 1),
        'q' : range(1,1+ 2),
        'sp' : range(1,1+ 1),
        'sd' : range(0,1+ 0),
        'sq' : range(1,1+ 1),
        's': [24]
    }
   
    auto_sarima(training_data=training_data, order_limits=order_limits, timesteps=timesteps, days=days, ruta=ruta)

In [ ]:
# Mejor modelo

# Parámetros
timesteps = 24
days = 31
df = sarima_model(training_data=training_data, order=(1,1,2), seasonal_order=(1,0,1,24), timesteps=timesteps, days=days)
df['mse'] = (df['nonshiftable']-df['forecast'])**2
df['mae'] = abs(df['nonshiftable']-df['forecast'])
df['mape'] = abs(df['nonshiftable']-df['forecast'])/df['nonshiftable']
mse_mean = df['mse'].mean()
mse_sd = df['mse'].var()**(1/2)
mae = df['mae'].mean()
mape = df['mape'].mean()
print('mse:', mse_mean)
print('mse_sd:',mse_sd)
print('mae',np.round(mae,2))
print('mape',np.round(mape,4))



In [21]:
dias = df['Day'].unique()
df['From'] = df['From'].astype(str)
df = df.reset_index(drop=True)
df.head()

,local_15min,nonshiftable,forecast,Upper bound,Lower bound,From,Day,mse,mae,mape
0,2019-06-01 00:00:00,0.055313,0.086360,0.213500,-0.040781,2019-05-31 23:00:00,2019-06-01,0.000964,0.031047,0.561304
1,2019-06-01 01:00:00,0.074063,0.093332,0.236297,-0.049633,2019-05-31 23:00:00,2019-06-01,0.000371,0.019270,0.260183
2,2019-06-01 02:00:00,0.049875,0.110693,0.257478,-0.036092,2019-05-31 23:00:00,2019-06-01,0.003699,0.060818,1.219408
3,2019-06-01 03:00:00,0.112375,0.125111,0.272876,-0.022653,2019-05-31 23:00:00,2019-06-01,0.000162,0.012736,0.113339
4,2019-06-01 04:00:00,0.137750,0.201356,0.349379,0.053333,2019-05-31 23:00:00,2019-06-01,0.004046,0.063606,0.461748


In [23]:
df_real= df[['local_15min', 'nonshiftable']].drop_duplicates()

In [39]:
df["From2"] = "Forecast at " + df["From"].str.slice(start=-8, stop=-3)

DIAS = dias[3:4]

colorscale = []
rgb = px.colors.convert_colors_to_same_type(px.colors.sequential.haline)[0]
n_steps = int(np.round(28/(len(rgb)-1)))  # Control the number of colors in the final colorscale
for i in range(len(rgb) - 1):
    for step in np.linspace(0, 1, n_steps):
        colorscale.append(px.colors.find_intermediate_color(rgb[i], rgb[i + 1], step, colortype='rgb'))

for i in range(len(colorscale)):
    colorscale[i] = colorscale[i][:-1]+', 0.5)'
    colorscale[i] = 'rgba'+colorscale[i][3:]

fig = px.line(df[df['Day'].isin(DIAS)], x="local_15min", y="forecast", color='From2',color_discrete_sequence=colorscale, labels={"From2": ""})

labels_to_show_in_legend = ["Forecast at 0{}:00".format(i) for i in range(0, 10, 2)]+["Forecast at {}:00".format(i) for i in range(10, 24, 2)]

for trace in fig['data']: 
    if (not trace['name'] in labels_to_show_in_legend):
        trace['showlegend'] = False

fig.add_trace(go.Scatter(x=df_real[df_real['local_15min'].dt.date.isin(pd.to_datetime(DIAS).date)]["local_15min"], 
                         y=df_real[df_real['local_15min'].dt.date.isin(pd.to_datetime(DIAS).date)]["nonshiftable"], 
                         name="Real energy demand",
                         line=dict(color='black'),))

fig.update_layout(title_text="Rolling forecast of non-shiftable energy demand",
                  yaxis=dict(title="Energy [kWh]"),
                  xaxis=dict(title= "Time"))

fig.write_image("../../Soportes documento/Graficas documento/demand_forecast.png", scale=2, height = 450, width=700)
fig.show()